In [1]:
import csv
import codecs
import collections
import nltk

tokenizer = nltk.tokenize.TweetTokenizer()

In [2]:
# ['tags', 'text', 'title', 'topic', 'url']
tags = [' спецпроект',
 'Coцсети',
 'Авто',
 'Автобизнес',
 'Аналитика рынка',
 'Архитектура',
 'Банки',
 'Баскетбол',
 'Безопасность ',
 'Белоруссия',
 'Бокс и ММА',
 'Вещи',
 'Вирусные ролики',
 'Вкусы',
 'Внешний вид',
 'Вооружение',
 'Вооруженные силы',
 'Все',
 'Выборы',
 'Гаджеты',
 'Госрегулирование',
 'Госэкономика',
 'Движение',
 'Деловой климат',
 'Деньги',
 'Достижения',
 'Еда',
 'Звери',
 'Зимние виды',
 'Игры',
 'Инновации',
 'Инструменты',
 'Интернет',
 'Искусство',
 'История',
 'Кавказ',
 'Казахстан',
 'Катастрофы',
 'Кино',
 'Книги',
 'Компании',
 'Конфликты',
 'Космос',
 'Криминал',
 'Крым',
 'Культпросвет',
 'Легпром',
 'Летние виды',
 'Люди',
 'Мемы',
 'Мир',
 'Мировой бизнес',
 'Мировой опыт',
 'Мнения',
 'Молдавия',
 'Москва',
 'Музыка',
 'Наследие',
 'Наука',
 'Общество',
 'Оружие',
 'Первая мировая',
 'Политика',
 'Полиция и спецслужбы',
 'Пресса',
 'Преступность',
 'Прибалтика',
 'Производители',
 'Происшествия',
 'Регионы',
 'Реклама',
 'Ресурсы',
 'Россия',
 'Рынки',
 'Следствие и суд',
 'События',
 'Средняя Азия',
 'Стиль',
 'Страноведение',
 'ТВ и радио',
 'Театр',
 'Теннис',
 'Техника',
 'Технологии',
 'Туризм',
 'Украина',
 'Финансы компаний',
 'Футбол',
 'Хоккей',
 'Часы',
 'Экология',
 'Явления']

num_tags = [{'tag': tag, 'id': i} for i, tag in enumerate(tags)]

texts = []
text_tags = []

with codecs.open('D:\Max\docs_cluster\lenta\lenta_data.csv', 'r', 'utf_8_sig') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        if i == 0:
            continue
        
        texts.append(row[1])
        text_tags.append(next((tag['id'] for tag in num_tags if tag['tag'] == row[0]), None))

In [4]:
'''
for i in range(5):
    print(texts[i])
    print(text_tags[i])
'''

corpus_size = len(texts)
selections_num = 6      # На сколько частей разбиваем датасет
selection_size = corpus_size // selections_num
print(selection_size)
print(corpus_size - (selections_num - 1) * selection_size)

'''
for i in range(selections_num - 1):
    start = i * selection_size
    end = start + selection_size
    print(str(start) + ' - ' + str(end))
'''

105936
105938


"\nfor i in range(selections_num - 1):\n    start = i * selection_size\n    end = start + selection_size\n    print(str(start) + ' - ' + str(end))\n"

Тренируемся на первых пяти датасетах без лемматизации, проверяемся на шестом

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

tfidf_transformer = TfidfTransformer()
count_vect = CountVectorizer()
lin_reg_models = []
multinom_models = []

for i in range(selections_num - 1):
    print("Selection #" + str(i))
    start = i * selection_size
    end = start + selection_size
    X_train_counts = count_vect.fit_transform(texts[start:end])
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    print("Training logistic regression")
    lin_reg_models.append(LogisticRegression().fit(X_train_tfidf, text_tags[start:end]))
    #print("Training multinomialNB")
    #multinom_models.append(MultinomialNB().fit(X_train_tfidf, text_tags[start:end]))

Selection #0
Training logistic regression
Selection #1
Training logistic regression
Selection #2
Training logistic regression
Selection #3
Training logistic regression
Selection #4
Training logistic regression


In [10]:
log_reg_hit = 0     #сколько раз классификатор угадал при проверке
multinom_hit = 0


start = (selections_num - 1) * selection_size
test_selection_cnt = corpus_size - (selections_num - 1) * selection_size
test_tags = text_tags[start:]

X_test_counts = count_vect.fit_transform(texts[start:])
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)

lin_reg_predictions = [model.predict(X_test_tfidf) for model in lin_reg_models]
multinom_predictions = [model.predict(X_test_tfidf) for model in multinom_models]

for i in range(7):
    predictions = [p[i] for p in lin_reg_predictions]
    prediction = max(set(predictions), key=lst.count)
    if test_tags[i] == prediction:
        log_reg_hit += 1


log_reg_accuracy = log_reg_hit / test_selection_cnt
print(log_reg_accuracy)

ValueError: X has 440699 features per sample; expecting 442020

# Ниже черновик

In [44]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, twenty_train.target)